In [ ]:
!pip install geojson sentinelhub numba

In [ ]:
# Import block
import numpy as np

np.set_printoptions(
    linewidth=160, precision=2, suppress=True, floatmode="maxprec_equal"
)
import matplotlib.pyplot as plt

from numba import njit, prange
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

# Remove annoying deprecation warnings
warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)

from enum import IntEnum
from PIL import Image, ImageOps

import geojson
from sentinelhub import (
    DataCollection,
    SHConfig,
    SentinelHubRequest,
    MimeType,
    bbox_to_dimensions,
    Geometry,
    BBoxSplitter,
    BBox,
    CRS,
)

import glob, os
from tqdm import tqdm, trange

import csv

In [ ]:
# Bands constants
class Bands(IntEnum):
    L = 1  # Grayscale
    LA = 2  # Grayscale w/Alpha
    RGB = 3  # Red, Green, Blue
    RGBA = 4  # Red, Green, Blue, Alpha


# Dataset constants
class Dataset:
    NDVI = "NDVI"
    TRUECOLOR = "TRUECOLOR"
    SWIR = "SWIR"


# Returns the sentinelrequest evalscript stirng to download 
# the specified dataset
def evalscript(dataset, bands):
    if dataset == Dataset.NDVI:
        return """
            //VERSION=3

            let viz = ColorMapVisualizer.createDefaultColorMap();

            function evaluatePixel(samples) {
                let val = index(samples.B08, samples.B04);
                val = viz.process(val);
                val.push(samples.dataMask);
                return val;
            } 

            function setup() {
                return {
                input: [{
                    bands: [
                    "B04",
                    "B08",
                    "dataMask"
                    ]
                }],
                output: {
                    bands: %d }
                }
            }
            """ % (
            bands
        )
    elif dataset == Dataset.TRUECOLOR:
        return """
            //VERSION=3
            let minVal = 0.0;
            let maxVal = 0.4;

            let viz = new HighlightCompressVisualizer(minVal, maxVal);

            function setup() {
            return {
                input: ["B04", "B03", "B02","dataMask"],
                output: { bands: %d }
            };
            }

            function evaluatePixel(samples) {
                let val = [samples.B04, samples.B03, samples.B02,samples.dataMask];
                return viz.processList(val);
            }""" % (
            bands
        )
    elif dataset == Dataset.SWIR:
        return """
            //VERSION=3
            let minVal = 0.0;
            let maxVal = 0.4;

            let viz = new HighlightCompressVisualizer(minVal, maxVal);

            function setup() {
            return {
                input: ["B12", "B8A", "B04","dataMask"],
                output: { bands: %d }
            };
            }

            function evaluatePixel(samples) {
                let val = [samples.B12, samples.B8A, samples.B04,samples.dataMask];
                return viz.processList(val);
            } """ % (
            bands
        )


In [ ]:
# Download image from sentinelhub, bounded by the border polygon,resolution given by bbox size
def download_image(
    bbox_size,
    border_polygon,
    data_folder,
    time_interval="latest",
    dataset=Dataset.TRUECOLOR,
    bands=Bands.RGBA,
    bbox=None,
    format="png",
    save_data=True,
    redownload=False,
):
    # User ID
    CLIENT_ID = "put-your-sentinel-hub-id-here"
    CLIENT_SECRET = "put-your-sentinel-hub-id-here"

    config = SHConfig()

    if CLIENT_ID and CLIENT_SECRET:
        config.sh_client_id = CLIENT_ID
        config.sh_client_secret = CLIENT_SECRET

    if config.sh_client_id == "" or config.sh_client_secret == "":
        print(
            "Warning! To use Sentinel Hub services, please provide the credentials (client ID and client secret)."
        )

    # Set image format, png has smaller file size than tiff
    img_format = MimeType.PNG
    if format == "tiff":
        img_format = MimeType.TIFF

    # Find latest data with low cloud coverage
    input_data = [
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            maxcc=0.01,
        )
    ]
    # Find data in specified date interval
    if time_interval != "latest":
        input_data = [
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=time_interval,
            )
        ]

    # Make the data request
    sentinel_request = SentinelHubRequest(
        evalscript=evalscript(dataset, bands),
        input_data=input_data,
        responses=[
            SentinelHubRequest.output_response("default", img_format),
        ],
        bbox=bbox,
        size=bbox_size,
        geometry=border_polygon,
        config=config,
        data_folder=data_folder,
    )
    # Return the data of the request, use cached if available
    return sentinel_request.get_data(save_data=save_data, redownload=redownload)[0]


In [ ]:
# Get the image of the area specified by the geojson with the given filename.
# The image may be split in tiles due to large size but can be stitched to one image.
def get_image(
    filename,
    dataset=Dataset.TRUECOLOR,
    bands=Bands.RGBA,
    time_interval="",
    stitch_tiles=False,
    format="png",
    force_download=False,
    debug=False,
):
    # Extract city name
    city_name = filename.split("/")[-1].split(".json")[0]    
    # Look for existing image, make its folder if missing
    data_folder = "Images/{:s}/{:s} {:s} {:n}".format(
        city_name, city_name, dataset, bands
    )

    stitched_image_filename = "Images/{:s}/{:s} {:s} {:n}.{:s}".format(
        city_name, city_name, dataset, bands, format
    )
    if stitch_tiles and os.path.exists(stitched_image_filename) and not force_download:
        return Image.open(stitched_image_filename)

    # Open the geojson file and extract the polygon(s)
    with open(filename) as file:
        shapefile = geojson.load(file)
        if "geometries" in shapefile.keys():
            polygon = shapefile["geometries"][0]
        else:
            polygon = shapefile

        # Find a specified time interval, if there is one
        if "timeInterval" in shapefile.keys():
            time_interval = (
                shapefile["timeInterval"]["startTime"],
                shapefile["timeInterval"]["endTime"],
            )
        elif time_interval == "":
            time_interval = "latest"

        # Make the bounding box (bbox) of the polygon
        city_geometry = Geometry.from_geojson(polygon, crs=CRS.WGS84)
        city_bbox = city_geometry.bbox
        # Figure out the image dimensions for 10 meters per pixel
        city_bbox_size = bbox_to_dimensions(city_bbox, 10)

    # Download a single image if the dimensions are small enough
    if not (city_bbox_size[0] > 2500 or city_bbox_size[1] > 2500):
        if debug:
            print(city_bbox_size)
        # Download the image/load it from cache
        data = download_image(
            city_bbox_size,
            city_geometry,
            data_folder,
            time_interval=time_interval,
            dataset=dataset,
            bands=bands,
            format=format,
            redownload=force_download,
        )
        image = Image.fromarray(data)
        if stitch_tiles:
            # Save the image with a more descriptive filename, easier acess
            image.save(stitched_image_filename)
        return image

    # If the dimensions are too large: Divide the image into a grid of tiles, and download
    else:
        if debug:
            print("Too large: ", city_bbox_size)
        # Find the number of tiles in both directions, add small buffer to size in division
        height = int(np.ceil(city_bbox_size[0] / 2480))
        width = int(np.ceil(city_bbox_size[1] / 2480))
        if debug:
            print("Splitting to {:n}x{:n} grid.".format(width, height))

        # Split the boundary box into a grid
        bboxes = city_geometry.bbox.get_partition(num_x=height, num_y=width)
        if debug:
            print(bboxes)

        # Because the dimensions of the tiles might vary by a few pixels (~10-20) because of the map projection,
        # we find the average tile dimensions. This is done to make the stitching
        # of the tiles seamless. The distortion (difference in resolution) is negligible, as we're
        # typically dealing with less than 1% difference from 10 m.
        cumulative_dim = [0, 0]
        for i, bbox_col in enumerate(bboxes):
            for j, bbox in enumerate(bbox_col):
                dims = bbox_to_dimensions(bbox, 10)
                cumulative_dim[0] += dims[0]
                cumulative_dim[1] += dims[1]
                if debug:
                    print(dims)
        num_bbox = (i + 1) * (j + 1)
        avg_dim = (int(cumulative_dim[0] / num_bbox), int(cumulative_dim[1] / num_bbox))
        if debug:
            print(avg_dim)

        if stitch_tiles:        
            # Prepare the array for the stitched tiles
            if bands == 1:
                stitched_image = np.full(
                    (avg_dim[1] * width, avg_dim[0] * height),
                    fill_value=np.nan,
                    dtype=np.uint8,
                )
            else:
                stitched_image = np.full(
                    (avg_dim[1] * width, avg_dim[0] * height, bands),
                    fill_value=np.nan,
                    dtype=np.uint8,
                )

        # List to contain the separate images, this will be returned if stitching is disabled
        images = []
        # Set up progressbar to keep track of the downloads
        pbar = tqdm(
            total=width * height,
            desc="Downloading {:s} {:s}".format(city_name, dataset),
            leave=False,
        )
        # Go through each bbox and download the image
        for x, row in enumerate(bboxes):
            for y, bbox in enumerate(row):
                if debug:
                    print(y, x)
                tile = download_image(
                    avg_dim,
                    city_geometry,
                    data_folder,
                    time_interval=time_interval,
                    dataset=dataset,
                    bands=bands,
                    bbox=bbox,
                    format=format,
                    redownload=force_download,
                )
                images.append(Image.fromarray(tile))
                if stitch_tiles:
                    # Add the tile to its corresponding place in the large stitched array
                    if bands == 1:
                        stitched_image[
                            (width - 1 - y) * avg_dim[1] : (width - y) * avg_dim[1],
                            (x) * avg_dim[0] : (x + 1) * avg_dim[0],
                        ] = tile
                    else:
                        stitched_image[
                            (width - 1 - y) * avg_dim[1] : (width - y) * avg_dim[1],
                            (x) * avg_dim[0] : (x + 1) * avg_dim[0],
                            :,
                        ] = tile
                pbar.update(1)
        pbar.close()

        if stitch_tiles:
            # Save the stitched image, set the returning images to the stitched one
            images = Image.fromarray(stitched_image)
            images.save(stitched_image_filename)
        return images


In [ ]:
# Highlights the vegetation in the true color image, also returns the vegetation fraction
def highlight_vegetation(
    image_NDVI, image_TRUECOLOR, bands=Bands.RGBA, highlight_color=(0, 255, 0, 255)
):
    # Convert images to arrays that numba can do calculations on, PIL images directly don't work
    shape = (image_NDVI.size[1], image_NDVI.size[0], bands)
    data_NDVI = np.reshape(np.array(image_NDVI.getdata()), shape)
    data_TRUECOLOR = np.reshape(np.array(image_TRUECOLOR.getdata()), shape)
    # Get the resulting array and vegetation fraction from the calculation function
    array, veg_frac, weight = highlight_vegetation_calc(
        data_NDVI, data_TRUECOLOR, bands=bands
    )
    # Convert back to image and return it with the fraction
    return Image.fromarray(np.array(array, dtype=np.uint8)), veg_frac, weight


# Higher performance calculation function, utilizes numba to speed up, but is stricter on types
@njit
def highlight_vegetation_calc(
    image_NDVI, image_TRUECOLOR, bands=Bands.RGBA, highlight_color=(0, 255, 0, 255)
):
    (height, width, _bands) = np.shape(image_NDVI)

    # Colors in NDVI that corresponds to vegetation
    green_encodings = [
        (51, 204, 204, 255),
        (0, 102, 102, 255),
        (51, 255, 51, 255),
        (51, 204, 51, 255),
        (0, 102, 0, 255),
    ]

    # Color that corresponds to water
    water_encoding = (0, 0, 0, 255)

    # Pixel counters to calculate vegetation fraction
    total_valid_pixels = 0
    total_green_pixels = 0

    # Go through each pixel in image and highlight if vegetation
    for y in range(height):
        for x in range(width):
            pixel = image_NDVI[y, x]
            # Make a comparable color tuple
            pixel_tuple = (pixel[0], pixel[1], pixel[2], pixel[3])
            if pixel_tuple in green_encodings:
                # Change color of pixel to the highlight color, increase both counters
                image_TRUECOLOR[y, x] = highlight_color
                total_green_pixels += 1
                total_valid_pixels += 1
            elif pixel_tuple[-1] != 0 and pixel_tuple != water_encoding:
                # If color is land, but not vegetation, increase total valid counter
                total_valid_pixels += 1

    # Return the highlighted image array, the vegetation fraction and the tile weight
    if total_valid_pixels != 0:
        return (
            image_TRUECOLOR,
            total_green_pixels / total_valid_pixels,
            total_valid_pixels,
        )
    return image_TRUECOLOR, 0, 0

In [ ]:
# Filters out everything except vegetation from the truecolor image
def vegetation_only(
    image_NDVI, image_TRUECOLOR, bands=Bands.RGBA, highlight_color=(0, 0, 0, 0)
):
    # Convert images to arrays that numba can do calculations on, PIL images directly don't work
    shape = (image_NDVI.size[1], image_NDVI.size[0], bands)
    data_NDVI = np.reshape(np.array(image_NDVI.getdata()), shape)
    data_TRUECOLOR = np.reshape(np.array(image_TRUECOLOR.getdata()), shape)
    # Get the resulting array, vegetation fraction and tile weight from the calculation function
    array, veg_frac, weight = vegetation_only_calc(
        data_NDVI, data_TRUECOLOR, bands=bands, highlight_color=highlight_color
    )
    # Convert back to image and return it with the fraction and weight
    return Image.fromarray(np.array(array, dtype=np.uint8)), veg_frac, weight


# Higher performance calculation function, utilizes numba to speed up, but is stricter on types
@njit
def vegetation_only_calc(
    image_NDVI, image_TRUECOLOR, bands=Bands.RGBA, highlight_color=(0, 0, 0, 0)
):
    height, width, bands = np.shape(image_NDVI)

    # Colors in NDVI that corresponds to vegetation
    green_encodings = [
        (51, 204, 204, 255),
        (0, 102, 102, 255),
        (51, 255, 51, 255),
        (51, 204, 51, 255),
        (0, 102, 0, 255),
    ]

    # Color that corresponds to water
    water_encoding = (0, 0, 0, 255)

    # Pixel counters to calculate vegetation fraction
    total_valid_pixels = 0
    total_green_pixels = 0

    # Go through each pixel in image and make transparent if not vegetation
    for y in range(height):
        for x in range(width):
            pixel = image_NDVI[y, x]
            # Make a comparable color tuple
            pixel_tuple = (pixel[0], pixel[1], pixel[2], pixel[3])
            if pixel_tuple not in green_encodings and pixel_tuple[-1] != 0:
                # Change color of pixel to the highlight color (transparent), increase total valid counter
                image_TRUECOLOR[y, x] = highlight_color
                total_valid_pixels += 1
            elif pixel_tuple in green_encodings:
                # If color is vegetation, increase both counters
                total_green_pixels += 1
                total_valid_pixels += 1
            elif pixel_tuple[-1] != 0 and pixel_tuple != water_encoding:
                # If color not transparent or water
                total_valid_pixels += 1

    # Return the highlighted image array, vegetation fraction and the tile weight
    if total_valid_pixels != 0:
        return (
            image_TRUECOLOR,
            total_green_pixels / total_valid_pixels,
            total_valid_pixels,
        )

    return image_TRUECOLOR, 0, 0

In [ ]:
# Filters out everything except vegetation, and highlights the vegetation, also returns vegetation
# fraction and tile weight
def vegetation_only_highlighted(
    image_NDVI, bands=Bands.RGBA, highlight_color=(0, 255, 0, 255)
):
    # Convert images to arrays that numba can do calculations on, PIL images directly don't work
    shape = (image_NDVI.size[1], image_NDVI.size[0], bands)
    data_NDVI = np.reshape(np.array(image_NDVI.getdata()), shape)
    # Get the resulting array, vegetation fraction and tile weight from the calculation function
    array, veg_frac, weight = vegetation_only_highlighted_calc(
        data_NDVI, bands=bands, highlight_color=highlight_color
    )
    # Convert back to image and return it with the fraction and weight
    return Image.fromarray(np.array(array, dtype=np.uint8)), veg_frac, weight


# Higher performance calculation function, utilizes numba to speed up, but is stricter on types
@njit
def vegetation_only_highlighted_calc(
    image_NDVI, bands=Bands.RGBA, highlight_color=(0, 255, 0, 255)
):
    height, width, bands = np.shape(image_NDVI)

    # Colors in NDVI that corresponds to vegetation
    green_encodings = [
        (51, 204, 204, 255),
        (0, 102, 102, 255),
        (51, 255, 51, 255),
        (51, 204, 51, 255),
        (0, 102, 0, 255),
    ]

    # Color that corresponds to water
    water_encoding = (0, 0, 0, 255)

    # Transparent color
    transparent = (0, 0, 0, 0)

    # Pixel counters to calculate vegetation fraction
    total_valid_pixels = 0
    total_green_pixels = 0

    # Go through each pixel in image and make transparent if not vegetation
    for y in range(height):
        for x in range(width):
            pixel = image_NDVI[y, x]
            # Make a comparable color tuple
            pixel_tuple = (pixel[0], pixel[1], pixel[2], pixel[3])
            if pixel_tuple in green_encodings:
                # If color is vegetation, increase both counters,
                # highlight the vegetation
                image_NDVI[y, x] = highlight_color
                total_green_pixels += 1
                total_valid_pixels += 1
            elif pixel_tuple[-1] != 0 and pixel_tuple != water_encoding:
                # If color not transparent or water, it's land
                image_NDVI[y, x] = transparent
                total_valid_pixels += 1
            else:
                # Filter out other water
                image_NDVI[y, x] = transparent

    # Return the highlighted image array, vegetation fraction and the tile weight
    if total_valid_pixels != 0:
        return (
            image_NDVI,
            total_green_pixels / total_valid_pixels,
            total_valid_pixels,
        )

    return image_NDVI, 0, 0

In [ ]:
# Calculate the vegetation fraction and tile weight, but no returning image
def veg_frac_only(image_NDVI, bands=Bands.RGBA):
    # Convert image to an array that numba can do calculations on, PIL images directly don't work
    shape = (image_NDVI.size[1], image_NDVI.size[0], bands)
    data_NDVI = np.reshape(np.array(image_NDVI.getdata()), shape)
    # Get the vegetation fraction and tile weight from the calculation function
    veg_frac, weight = veg_frac_only_calc(data_NDVI)
    return veg_frac, weight


# Higher performance calculation function, utilizes numba to speed up, but is stricter on types
@njit
def veg_frac_only_calc(image_NDVI):
    height, width, bands = np.shape(image_NDVI)

    # Colors in NDVI that corresponds to vegetation
    green_encodings = [
        (51, 204, 204, 255),
        (0, 102, 102, 255),
        (51, 255, 51, 255),
        (51, 204, 51, 255),
        (0, 102, 0, 255),
    ]

    # Color that corresponds to water
    water_encoding = (0, 0, 0, 255)

    # Pixel counters to calculate vegetation fraction
    total_valid_pixels = 0
    total_green_pixels = 0

    # Go through each pixel in image to find vegetation
    for y in range(height):
        for x in range(width):
            pixel = image_NDVI[y, x]
            # Make a comparable color tuple
            pixel_tuple = (pixel[0], pixel[1], pixel[2], pixel[3])
            if pixel_tuple in green_encodings:
                # Color is vegetation, increase both counters
                total_green_pixels += 1
                total_valid_pixels += 1
            elif pixel_tuple[-1] != 0 and pixel_tuple != water_encoding:
                # If color not transparent or water, it's land
                total_valid_pixels += 1

    # Return vegetation fraction and tile weight
    if total_valid_pixels != 0:
        return total_green_pixels / total_valid_pixels, total_valid_pixels

    return 0, 0

In [ ]:
# Decide whether to run through all the cities
run = False
if run:
    # Map to store cities with their vegetation fraction
    cities = {}
    # Get all cities in the path
    city_list = sorted(glob.glob("American Cities/*"))

    # Run through all the cities, with progressbar
    for city in tqdm(city_list, desc="Progress"):
        # Extract city name from filename
        city_name = city.split("/")[-1].split(".json")[0]
        # Get the NDVI image for the city
        image_ndvi = get_image(
            city, dataset=Dataset.NDVI, bands=Bands.RGBA, stitch_tiles=False
        )

        # Check if we got a list of image tiles or a single image
        if not isinstance(image_ndvi, list):
            image_veg_only_hl, veg_frac, weight = vegetation_only_highlighted(
                image_ndvi
            )
            cities[city_name] = veg_frac
        else:
            # If we got a list of image tiles, we need lists to be able to
            # weight the results
            fracs = []
            weights = []
            # Go through each image, with progressbar
            for image in tqdm(
                image_ndvi,
                total=len(image_ndvi),
                leave=False,
                desc="Processing {:s}".format(city_name),
            ):
                image_veg_only_hl, veg_frac, weight = vegetation_only_highlighted(image)
                fracs.append(veg_frac)
                weights.append(weight)

            # Find the total weighted vegetation fraction
            frac = np.sum(np.multiply(weights, fracs) / np.sum(weights))
            cities[city_name] = frac

        # Write result to temporary result file, so we don't overwrite the previous final file 
        # if we spot that something is wrong
        with open("result_tmp.csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile, delimiter=",")
            writer.writerow(["City", "Vegetation Fraction"])
            for key in cities:
                writer.writerow([key, cities[key]])
    
    # Loop is done, write final result to file
    with open("result.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(["City", "Vegetation Fraction"])
        for key in cities:
            writer.writerow([key, cities[key]])